In [1]:
import datajoint as dj

## Connect to DB

In [2]:
dj.config['database.host'] = '127.0.0.1'
dj.config['database.user'] = 'root'
dj.config['database.password'] = 'tutorial'
schema = dj.schema('tutorial', locals())

Connecting root@127.0.0.1:3306


## Define and create table

In [3]:
@schema
class Mouse(dj.Manual):
      definition = """
      # mouse
      mouse_id: int                  # unique mouse id
      ---
      dob: date                      # mouse date of birth
      sex: enum('M', 'F', 'U')    # sex of mouse - Male, Female, or Unknown/Unclassified
      """

In [4]:
mouse = Mouse()
mouse

mouse_id unique mouse id,dob mouse date of birth,"sex sex of mouse - Male, Female, or Unknown/Unclassified"


## Populate table

In [5]:
mouse.insert1((0, '2017-03-01', 'M') )

In [6]:
data = {
  'mouse_id': 100,
  'dob': '2017-05-12',
  'sex': 'F'
}

In [7]:
mouse.insert1(data)

In [8]:
mouse

mouse_id unique mouse id,dob mouse date of birth,"sex sex of mouse - Male, Female, or Unknown/Unclassified"
0,2017-03-01,M
100,2017-05-12,F


In [9]:
data = [
  (1, '2016-11-19', 'M'),
  (2, '2016-11-20', 'U'),
  (5, '2016-12-25', 'F')
]

# now insert all at once
mouse.insert(data)

In [10]:
mouse

mouse_id unique mouse id,dob mouse date of birth,"sex sex of mouse - Male, Female, or Unknown/Unclassified"
0,2017-03-01,M
1,2016-11-19,M
2,2016-11-20,U
5,2016-12-25,F
100,2017-05-12,F


In [11]:
# tuples or dicts possible
data = [
  {'mouse_id': 10, 'dob': '2017-01-01', 'sex': 'F'},
  {'mouse_id': 11, 'dob': '2017-01-03', 'sex': 'F'},
]

# insert them all
mouse.insert(data)

In [12]:
mouse

mouse_id unique mouse id,dob mouse date of birth,"sex sex of mouse - Male, Female, or Unknown/Unclassified"
0,2017-03-01,M
1,2016-11-19,M
2,2016-11-20,U
5,2016-12-25,F
10,2017-01-01,F
11,2017-01-03,F
100,2017-05-12,F


## Persist changes
place table definitions in a file and import

In [13]:
from src.tutorial_tables import *

## Querying the table 

In [14]:
data = mouse.fetch()

In [15]:
type(data)

numpy.ndarray

In [16]:
data

array([(  0, datetime.date(2017, 3, 1), 'M'),
       (  1, datetime.date(2016, 11, 19), 'M'),
       (  2, datetime.date(2016, 11, 20), 'U'),
       (  5, datetime.date(2016, 12, 25), 'F'),
       ( 10, datetime.date(2017, 1, 1), 'F'),
       ( 11, datetime.date(2017, 1, 3), 'F'),
       (100, datetime.date(2017, 5, 12), 'F')],
      dtype=[('mouse_id', '<i8'), ('dob', 'O'), ('sex', 'O')])

In [17]:
data_dict = mouse.fetch(as_dict=True)

In [18]:
type(data_dict)

list

In [19]:
data_dict

[{'mouse_id': 0, 'dob': datetime.date(2017, 3, 1), 'sex': 'M'},
 {'mouse_id': 1, 'dob': datetime.date(2016, 11, 19), 'sex': 'M'},
 {'mouse_id': 2, 'dob': datetime.date(2016, 11, 20), 'sex': 'U'},
 {'mouse_id': 5, 'dob': datetime.date(2016, 12, 25), 'sex': 'F'},
 {'mouse_id': 10, 'dob': datetime.date(2017, 1, 1), 'sex': 'F'},
 {'mouse_id': 11, 'dob': datetime.date(2017, 1, 3), 'sex': 'F'},
 {'mouse_id': 100, 'dob': datetime.date(2017, 5, 12), 'sex': 'F'}]

In [20]:
sex, ids = mouse.fetch('sex', 'mouse_id')

In [21]:
sex

array(['M', 'M', 'U', 'F', 'F', 'F', 'F'], dtype=object)

In [22]:
ids

array([  0,   1,   2,   5,  10,  11, 100])

In [23]:
mouse & 'mouse_id = 0'

mouse_id unique mouse id,dob mouse date of birth,"sex sex of mouse - Male, Female, or Unknown/Unclassified"
0,2017-03-01,M


In [24]:
mouse & 'sex = "M"'

mouse_id unique mouse id,dob mouse date of birth,"sex sex of mouse - Male, Female, or Unknown/Unclassified"
0,2017-03-01,M
1,2016-11-19,M


In [25]:
mouse & 'dob > "2017-01-01"' & 'sex = "M"'

mouse_id unique mouse id,dob mouse date of birth,"sex sex of mouse - Male, Female, or Unknown/Unclassified"
0,2017-03-01,M


In [26]:
type(mouse & 'dob > "2017-01-01"' & 'sex = "M"')

__main__.Mouse

In [32]:
# using a dict for the query
r = {
      'sex': 'M',
      'mouse_id' : 0
    }
mouse & r

mouse_id unique mouse id,dob mouse date of birth,"sex sex of mouse - Male, Female, or Unknown/Unclassified"
0,2017-03-01,M


In [34]:
# collect final result
(mouse & r).fetch()

array([(0, datetime.date(2017, 3, 1), 'M')],
      dtype=[('mouse_id', '<i8'), ('dob', 'O'), ('sex', 'O')])

In [35]:
# delete rows in the db
(mouse & 'mouse_id = 10').delete()

About to delete:
`tutorial`.`mouse`: 1 items
Proceed? [yes, No]: no
Cancelled deletes.
